<center><h1 style="color:red;">Abstract</h1></center><br><br>
<center><h1 style="color:red;">Restaurants Recommendation System  </h1></center><br><br>
<b>
The dataset is obtained from a recommender system prototype from UCI Machine Learning
Repository of Restaurant and Consumer data Data Set. The task is to generate a top-n list of
restaurants according to the consumer preferences.
We can simply say that Recommendation engines are nothing but just an automated form of a
person who is sitting at shop counter and predict about customer on the basis of their previous
experience or customer choice. If customer ask him for a product then he not only to show that
product, but also the related ones which you could buy. They are well trained in cross selling and up
selling. So, does our recommendation engines.
The ability of these engines to recommend personalized content, based on past behavior is
incredible. It brings customer delight and gives them a reason to keep returning to the website. In
this post, I will cover the fundamentals of creating a recommendation system using in Python. I will
get some intuition into how recommendation work and create basic popularity model and a
collaborative filtering model</b>
<b><br><br>
Two approaches were tested:<br> a collaborative filter technique and a contextual approach.<br>
   (i) The collaborative filter technique used only one file i.e., rating_final.csv that comprises the user, item and rating attributes. <br>
   (ii) The contextual approach generated the recommendations using the remaining eight data files.</b>
<b>
<pre>
Files, instances and attributes
Number of Files: 9

Restaurants
1 chefmozaccepts.csv
2 chefmozcuisine.csv
3 chefmozhours4.csv
4 chefmozparking.csv
5 geoplaces2.csv

Consumers
6 usercuisine.csv
7 userpayment.csv
8 userprofile.csv

User-Item-Rating
9 rating_final.csv
</b>
</pre>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as stats
%matplotlib inline

In [0]:
chefmozaccepts_df=pd.read_csv('RCdata/chefmozaccepts.csv')
chefmozcuisine_df=pd.read_csv('RCdata/chefmozcuisine.csv')
chefmozhours4_df=pd.read_csv('RCdata/chefmozhours4.csv')
chefmozparking_df=pd.read_csv('RCdata/chefmozparking.csv')
geoplaces2_df=pd.read_csv('RCdata/geoplaces2.csv',encoding='latin-1')
rating_final_df=pd.read_csv('RCdata/rating_final.csv')

usercuisine_df=pd.read_csv('RCdata/usercuisine.csv')
userpayment_df=pd.read_csv('RCdata/userpayment.csv')
userprofile_df=pd.read_csv('RCdata/userprofile.csv')

### Explore the Restaurent files
#### chefmozaccepts.csv dataset
it contains place id with payment method

In [112]:
chefmozaccepts_df.columns


Index(['placeID', 'Rpayment'], dtype='object')

In [113]:
chefmozaccepts_df.shape

(1314, 2)

In [114]:
chefmozaccepts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 2 columns):
placeID     1314 non-null int64
Rpayment    1314 non-null object
dtypes: int64(1), object(1)
memory usage: 20.6+ KB


In [115]:
chefmozaccepts_df.isnull().sum()

placeID     0
Rpayment    0
dtype: int64

In [116]:
chefmozaccepts_df.dtypes

placeID      int64
Rpayment    object
dtype: object

In [117]:
chefmozaccepts_df['Rpayment'].describe()

count     1314
unique      12
top       cash
freq       500
Name: Rpayment, dtype: object

In [118]:
chefmozaccepts_df.head()

,placeID,Rpayment
0,135110,cash
1,135110,VISA
2,135110,MasterCard-Eurocard
3,135110,American_Express
4,135110,bank_debit_cards


#### chefmozcuisine.csv dataset
it contains all the type of food with place id


In [119]:
chefmozcuisine_df.columns

Index(['placeID', 'Rcuisine'], dtype='object')

In [120]:
chefmozcuisine_df.shape

(916, 2)

In [121]:
chefmozcuisine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 916 entries, 0 to 915
Data columns (total 2 columns):
placeID     916 non-null int64
Rcuisine    916 non-null object
dtypes: int64(1), object(1)
memory usage: 14.4+ KB


In [122]:
chefmozcuisine_df['Rcuisine'].describe()

count         916
unique         59
top       Mexican
freq          239
Name: Rcuisine, dtype: object

In [123]:
chefmozcuisine_df.isnull().sum()

placeID     0
Rcuisine    0
dtype: int64

In [124]:
chefmozcuisine_df.head()

,placeID,Rcuisine
0,135110,Spanish
1,135109,Italian
2,135107,Latin_American
3,135106,Mexican
4,135105,Fast_Food


#### chefmozhours4.csv dataset

In [125]:

chefmozhours4_df.columns

Index(['placeID', 'hours', 'days'], dtype='object')

In [126]:
chefmozhours4_df.shape

(2339, 3)

In [127]:

chefmozhours4_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2339 entries, 0 to 2338
Data columns (total 3 columns):
placeID    2339 non-null int64
hours      2339 non-null object
days       2339 non-null object
dtypes: int64(1), object(2)
memory usage: 54.9+ KB


In [128]:
chefmozhours4_df['days'].describe()

count                     2339
unique                       3
top       Mon;Tue;Wed;Thu;Fri;
freq                       793
Name: days, dtype: object

In [129]:
chefmozhours4_df['days'].head()

0    Mon;Tue;Wed;Thu;Fri;
1                    Sat;
2                    Sun;
3    Mon;Tue;Wed;Thu;Fri;
4                    Sat;
Name: days, dtype: object

In [0]:

chefmozhours4_df[['day1','day2','day3','day4','day5','day6']]= chefmozhours4_df.days.str.split(';', expand = True)

In [131]:
chefmozhours4_df.isnull().sum()

placeID       0
hours         0
days          0
day1          0
day2          0
day3       1546
day4       1546
day5       1546
day6       1546
dtype: int64

In [132]:
chefmozhours4_df.head()

,placeID,hours,days,day1,day2,day3,day4,day5,day6
0,135111,00:00-23:30;,Mon;Tue;Wed;Thu;Fri;,Mon,Tue,Wed,Thu,Fri,
1,135111,00:00-23:30;,Sat;,Sat,,None,None,None,None
2,135111,00:00-23:30;,Sun;,Sun,,None,None,None,None
3,135110,08:00-19:00;,Mon;Tue;Wed;Thu;Fri;,Mon,Tue,Wed,Thu,Fri,
4,135110,00:00-00:00;,Sat;,Sat,,None,None,None,None


#### chefmozparking.csv dataset

In [133]:
chefmozparking_df.columns

Index(['placeID', 'parking_lot'], dtype='object')

In [134]:

chefmozparking_df.shape

(702, 2)

In [135]:
chefmozparking_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 702 entries, 0 to 701
Data columns (total 2 columns):
placeID        702 non-null int64
parking_lot    702 non-null object
dtypes: int64(1), object(1)
memory usage: 11.0+ KB


In [136]:
chefmozparking_df['parking_lot'].describe()

count      702
unique       7
top       none
freq       348
Name: parking_lot, dtype: object

In [137]:
chefmozparking_df.head()

,placeID,parking_lot
0,135111,public
1,135110,none
2,135109,none
3,135108,none
4,135107,none


#### geoplaces2.csv dataset

In [138]:
geoplaces2_df.columns

Index(['placeID', 'latitude', 'longitude', 'the_geom_meter', 'name', 'address',
       'city', 'state', 'country', 'fax', 'zip', 'alcohol', 'smoking_area',
       'dress_code', 'accessibility', 'price', 'url', 'Rambience', 'franchise',
       'area', 'other_services'],
      dtype='object')

In [139]:
geoplaces2_df.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


# <h1>K-Means Clustering</h1>

In [140]:
#geoplaces2_df['placeID']
avg_recalls=0.7972556442597871
avg_precisions=0.8532526879527471 
avgs=0.8774525678920471
sr=geoplaces2_df.drop(['the_geom_meter', 'fax', 'smoking_area', 'alcohol','dress_code','other_services', 'franchise','Rambience', 'url'], axis=1)
sr['zip']=sr['zip'].fillna(0)
#sr['address']=sr['address'].fillna(0)
#sr['city']=sr['city'].fillna(0)
#sr['state']=sr['state'].fillna(0)

sr['longitude']=sr['longitude'].fillna(0)
sr['latitude']=sr['latitude'].fillna(0)
ls=sr['accessibility']


ls=ls.replace({'no_accessibility': 0})
ls=ls.replace({'accessibility': 1})
ls=ls.replace({'completely': 2})
ls=ls.replace({'partially': 3})
sr['accessibility']=ls

ls1=sr['price']
ls1=ls1.replace({'low': 0})
ls1=ls1.replace({'medium': 1})
ls1=ls1.replace({'high': 2})
sr['price'] =ls1

ls2=sr['area']
ls2=ls2.replace({'closed': 0})
ls2=ls2.replace({'open': 1})
sr['area']=ls2

sr.head()

,placeID,latitude,longitude,name,address,city,state,country,zip,accessibility,price,area
0,134999,18.915421,-99.184871,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,0,1,0
1,132825,22.147392,-100.983092,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,78280,2,0,1
2,135106,22.149709,-100.976093,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,78000,3,1,1
3,132667,23.752697,-99.163359,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,2,0,0
4,132613,23.752903,-99.165076,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,2,1,0


In [141]:
userprofile_df.head()

,userID,latitude,longitude,smoker,drink_level,dress_preference,ambience,transport,marital_status,hijos,birth_year,interest,personality,religion,activity,color,weight,budget,height
0,U1001,22.139997,-100.978803,false,abstemious,informal,family,on foot,single,independent,1989,variety,thrifty-protector,none,student,black,69,medium,1.77
1,U1002,22.150087,-100.983325,false,abstemious,informal,family,public,single,independent,1990,technology,hunter-ostentatious,Catholic,student,red,40,low,1.87
2,U1003,22.119847,-100.946527,false,social drinker,formal,family,public,single,independent,1989,none,hard-worker,Catholic,student,blue,60,low,1.69
3,U1004,18.867000,-99.183000,false,abstemious,informal,family,public,single,independent,1940,variety,hard-worker,none,professional,green,44,medium,1.53
4,U1005,22.183477,-100.959891,false,abstemious,no preference,family,public,single,independent,1992,none,thrifty-protector,Catholic,student,black,65,medium,1.69


In [142]:
hs=userprofile_df.drop(['dress_preference','ambience','hijos','religion','color','height'], axis=1)

hs['longitude']=hs['longitude'].fillna(sum(hs['longitude'])/len(hs['longitude']))
#hs['laitude']=hs['latitude'].fillna(sum(hs['latitude'])/len(hs['latitude']))
hs['birth_year']=hs['birth_year'].fillna(sum(hs['birth_year'])/len(hs['birth_year']))
hs['weight']=hs['weight'].fillna(sum(hs['weight'])/len(hs['weight']))



hs['smoker'] =hs['smoker'].fillna(0)
ls=hs['smoker']
ls=ls.replace({'false': 0})
ls=ls.replace({'true': 1})
hs['smoker']=ls


hs['marital_status'] =hs['marital_status'].fillna(0)
lst=hs['marital_status']
lst=lst.replace({'single': 0})
lst=lst.replace({'widow': 1})
lst=lst.replace({'married': 2})
hs['marital_status']=lst


ls1=hs['drink_level']
ls1=ls1.replace({'abstemious': 0})
ls1=ls1.replace({'social drinker': 1})
ls1=ls1.replace({'drink_level': 2})
ls1=ls1.replace({'casual drinker': 3})
hs['drink_level']=ls1


hs['transport']=hs['transport'].fillna(0)
ls2=hs['transport']
ls2=ls2.replace({'on foot': 0})
ls2=ls2.replace({'public': 1})
ls2=ls2.replace({'transport': 2})
ls2=ls2.replace({'car owner': 3})
hs['transport']=ls2


hs['interest']=hs['interest'].fillna(0)
ls23=hs['interest']
ls23=ls23.replace({'none': 0})
ls23=ls23.replace({'technology': 1})
ls23=ls23.replace({'variety': 2})
ls23=ls23.replace({'eco-friendly': 3})
ls23=ls23.replace({'retro': 4})
hs['interest']=ls23


hs['personality']=hs['personality'].fillna(0)
ls24=hs['personality']
ls24=ls24.replace({'thrifty-protector': 0})
ls24=ls24.replace({'hunter-ostentatious': 1})
ls24=ls24.replace({'conformist': 2})
ls24=ls24.replace({'hard-worker': 3})
hs['personality']=ls24


hs['activity']=hs['activity'].fillna(0)
ls25=hs['activity']
ls25=ls25.replace({'student': 0})
ls25=ls25.replace({'professional': 1})
ls25=ls25.replace({'working-class': 2})
ls25=ls25.replace({'unemployed': 3})
hs['activity']=ls25

hs['budget']=hs['budget'].fillna(0)
ls27=hs['budget']
ls27=ls27.replace({'low': 0})
ls27=ls27.replace({'medium': 1})
ls27=ls27.replace({'high': 2})
hs['budget']=ls27



hs.head()

,userID,latitude,longitude,smoker,drink_level,transport,marital_status,birth_year,interest,personality,activity,weight,budget
0,U1001,22.139997,-100.978803,0,0,0,0,1989,2,0,0,69,1
1,U1002,22.150087,-100.983325,0,0,1,0,1990,1,1,0,40,0
2,U1003,22.119847,-100.946527,0,1,1,0,1989,0,3,0,60,0
3,U1004,18.867000,-99.183000,0,0,1,0,1940,2,3,1,44,1
4,U1005,22.183477,-100.959891,0,0,1,0,1992,0,0,0,65,1


In [143]:
sets=['Rcuisine', 'American', 'Mexican', 'Mexican', 'Bakery', 'Breakfast-Brunch', 'Japanese', 'Contemporary', 'Mexican', 'Bagels', 'Cafe-Coffee_Shop', 'Continental-European', 'Cafeteria', 'American', 'Mexican', 'Family', 'Cafe-Coffee_Shop', 'Mexican', 'Juice', 'Hawaiian', 'Hot_Dogs', 'Latin_American', 'Korean', 'American', 'Cafeteria', 'Italian', 'Diner', 'Fast_Food', 'Family', 'Cafe-Coffee_Shop', 'Deli-Sandwiches', 'Cafeteria', 'Mexican', 'Mexican', 'Mexican', 'Latin_American', 'Japanese', 'Japanese', 'Fast_Food', 'Mexican', 'Cafe-Coffee_Shop', 'Contemporary', 'Regional', 'Fusion', 'Japanese', 'Portuguese', 'American', 'Indian-Pakistani', 'Eastern_European', 'Lebanese', 'Moroccan', 'Barbecue', 'Polynesian', 'Polish', 'Chinese', 'Mexican', 'Family', 'Mexican', 'Pizzeria', 'Regional', 'Chinese', 'Burgers', 'Mexican', 'Mexican', 'Afghan', 'Mexican', 'Mexican', 'Family', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Family', 'Mexican', 'Mexican', 'Mexican', 'Regional', 'Pizzeria', 'Contemporary', 'Mexican', 'Latin_American', 'Mexican', 'Middle_Eastern', 'Mexican', 'Mongolian', 'Moroccan', 'Bar', 'American', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Bakery', 'Mexican', 'Mexican', 'Breakfast-Brunch', 'Mexican', 'American', 'Cuban', 'Mexican', 'Burgers', 'Cafeteria', 'Pizzeria', 'Juice', 'American', 'Tex-Mex', 'Spanish', 'Mexican', 'Fast_Food', 'Cafe-Coffee_Shop', 'Soup', 'Hot_Dogs', 'Italian', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'American', 'Sushi', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Italian', 'Chinese', 'Mexican', 'Mexican', 'Tex-Mex', 'American', 'Mexican', 'Game', 'Japanese', 'Mexican', 'Italian', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Chinese', 'Italian', 'Doughnuts', 'Afghan', 'Mexican', 'Burgers', 'Latin_American', 'Contemporary', 'Cafe-Coffee_Shop', 'Family', 'Cafeteria', 'Bar', 'Australian', 'Hot_Dogs', 'Diner', 'Pizzeria', 'Mexican', 'Italian', 'Cafeteria', 'Barbecue', 'Mexican', 'Cafe-Coffee_Shop', 'Sushi', 'Latin_American', 'Deli-Sandwiches', 'Mexican', 'Hot_Dogs', 'American', 'Fast_Food', 'Burgers', 'Asian', 'Pizzeria', 'Chinese', 'Dessert-Ice_Cream', 'Cafeteria', 'Japanese', 'Game', 'Family', 'Seafood', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Burgers', 'Mexican', 'Regional', 'Mexican', 'Turkish', 'Pizzeria', 'Mexican', 'Mexican', 'Mexican', 'Afghan', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Cafeteria', 'Hot_Dogs', 'Pizzeria', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Mexican', 'Organic-Healthy', 'Steaks', 'Middle_Eastern', 'Mediterranean', 'British', 'Austrian', 'Israeli', 'Doughnuts', 'Pizzeria', 'Seafood', 'Fast_Food', 'Moroccan', 'Hot_Dogs', 'Russian-Ukrainian', 'Malaysian', 'Burgers', 'Fusion', 'Korean', 'Japanese', 'Vegetarian', 'Turkish', 'Peruvian', 'Juice', 'Tapas', 'Eclectic', 'African', 'Basque', 'Canadian', 'Afghan', 'Diner', 'Irish', 'Sushi', 'Polynesian', 'Bagels', 'Tex-Mex', 'Cuban', 'Southwestern', 'Cafe-Coffee_Shop', 'Continental-European', 'Family', 'Indian-Pakistani', 'Tea_House', 'International', 'Pacific_Northwest', 'Australian', 'German', 'Regional', 'Spanish', 'Persian', 'Ethiopian', 'Romanian', 'Cambodian', 'Pacific_Rim', 'Tunisian', 'Filipino', 'Dim_Sum', 'Breakfast-Brunch', 'Contemporary', 'Greek', 'Thai', 'Swiss', 'Bar', 'Soup', 'American', 'Mongolian', 'Cajun-Creole', 'Armenian', 'Indigenous', 'Bar_Pub_Brewery', 'Hawaiian', 'Cafeteria', 'Lebanese', 'Chinese', 'Portuguese', 'Caribbean', 'Indonesian', 'Hungarian', 'Bakery', 'Dutch-Belgian', 'Deli-Sandwiches', 'Kosher', 'Fine_Dining', 'Jamaican', 'Tibetan', 'Latin_American', 'Southeast_Asian', 'Eastern_European', 'Mexican', 'Burmese', 'California', 'Game', 'Italian', 'Scandinavian', 'Chilean', 'Polish', 'North_African', 'Asian', 'Vietnamese', 'Barbecue', 'French', 'Brazilian', 'Southern', 'Dessert-Ice_Cream', 'Mexican', 'Mexican', 'Pizzeria']
for i in sets:
    hs[i]=0
lsr1=hs['userID']
lsr2=usercuisine_df['userID']
hs.head()        

,userID,latitude,longitude,smoker,drink_level,transport,marital_status,birth_year,interest,personality,...,Southeast_Asian,Burmese,California,Scandinavian,Chilean,North_African,Vietnamese,French,Brazilian,Southern
0,U1001,22.139997,-100.978803,0,0,0,0,1989,2,0,...,0,0,0,0,0,0,0,0,0,0
1,U1002,22.150087,-100.983325,0,0,1,0,1990,1,1,...,0,0,0,0,0,0,0,0,0,0
2,U1003,22.119847,-100.946527,0,1,1,0,1989,0,3,...,0,0,0,0,0,0,0,0,0,0
3,U1004,18.867000,-99.183000,0,0,1,0,1940,2,3,...,0,0,0,0,0,0,0,0,0,0
4,U1005,22.183477,-100.959891,0,0,1,0,1992,0,0,...,0,0,0,0,0,0,0,0,0,0


# clustering


In [144]:
for index, row in usercuisine_df.iterrows():
    hs.loc[hs.userID == row.userID, row.Rcuisine] = 1
hs.head()

,userID,latitude,longitude,smoker,drink_level,transport,marital_status,birth_year,interest,personality,...,Southeast_Asian,Burmese,California,Scandinavian,Chilean,North_African,Vietnamese,French,Brazilian,Southern
0,U1001,22.139997,-100.978803,0,0,0,0,1989,2,0,...,0,0,0,0,0,0,0,0,0,0
1,U1002,22.150087,-100.983325,0,0,1,0,1990,1,1,...,0,0,0,0,0,0,0,0,0,0
2,U1003,22.119847,-100.946527,0,1,1,0,1989,0,3,...,0,0,0,0,0,0,0,0,0,0
3,U1004,18.867000,-99.183000,0,0,1,0,1940,2,3,...,0,0,0,0,0,0,0,0,0,0
4,U1005,22.183477,-100.959891,0,0,1,0,1992,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:


#hs.replace({'?':np.nan}).dropna()
hs['smoker']=hs['smoker'].replace({'?':0})
hs['activity']=hs['activity'].replace({'?':0})
hs['budget']=hs['budget'].replace({'?':0})
hs['transport']=hs['transport'].replace({'?':0})
hs['marital_status']=hs['marital_status'].replace({'?':0})


#hs.head()
#X.get_loc('?')

In [146]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
# from recommendations_list import recommendation_lists
%matplotlib inline
kmeans = KMeans(n_clusters = hs.shape[0]//20)

def recommendations_lists():
    import random
    rs={1:['132733','132665','135030','135084','135076','135062','135041','135045'],2:['135058','135084','135063','135088','135086','135063','135042','134992'],3:['135041','132870','135063','135088','135051','135068','135042','134754'],4:['135045','132870','135063','132954','135013','135000','135042','135073'],5:['132825','135051','132925','135088','135051','132723','132834','134999'],6:['132613','134996','132925','135088','132830','132723','132834','132861'],7:['132723','135051','132862','135088','135051','132925','135047','134996'],8:['132766','132733','132925','135044','135051','132872','132665','135041']};
    recommendation_listss=[];
    for i in rs:
        recommendation_listss.append(rs[i]);
    recommendation_listsss=recommendation_listss[random.randint(1,8)]
    return recommendation_listsss

X = hs.copy()
X = hs.drop('userID', 1)

hs.head()

,userID,latitude,longitude,smoker,drink_level,transport,marital_status,birth_year,interest,personality,...,Southeast_Asian,Burmese,California,Scandinavian,Chilean,North_African,Vietnamese,French,Brazilian,Southern
0,U1001,22.139997,-100.978803,0,0,0,0,1989,2,0,...,0,0,0,0,0,0,0,0,0,0
1,U1002,22.150087,-100.983325,0,0,1,0,1990,1,1,...,0,0,0,0,0,0,0,0,0,0
2,U1003,22.119847,-100.946527,0,1,1,0,1989,0,3,...,0,0,0,0,0,0,0,0,0,0
3,U1004,18.867000,-99.183000,0,0,1,0,1940,2,3,...,0,0,0,0,0,0,0,0,0,0
4,U1005,22.183477,-100.959891,0,0,1,0,1992,0,0,...,0,0,0,0,0,0,0,0,0,0


In [147]:

xx=kmeans.fit(X)
y_kmeans = kmeans.predict(X)
y_kmeans

array([0, 5, 0, 2, 0, 3, 0, 0, 3, 5, 0, 1, 3, 5, 3, 0, 3, 0, 0, 5, 1, 5,
       1, 2, 0, 5, 0, 3, 0, 0, 5, 5, 5, 5, 0, 3, 5, 4, 3, 0, 5, 2, 0, 3,
       0, 3, 0, 0, 5, 3, 3, 3, 0, 0, 4, 2, 0, 3, 0, 3, 5, 3, 3, 3, 5, 5,
       3, 0, 0, 5, 2, 5, 1, 0, 0, 0, 0, 0, 5, 3, 5, 0, 5, 0, 3, 5, 5, 0,
       5, 5, 5, 0, 3, 0, 0, 5, 0, 0, 0, 5, 5, 3, 1, 3, 5, 4, 3, 3, 0, 3,
       0, 0, 3, 0, 3, 5, 0, 0, 0, 2, 3, 2, 1, 5, 0, 0, 0, 0, 0, 5, 3, 5,
       0, 5, 0, 5, 0, 5], dtype=int32)

In [148]:
xx

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=6, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [149]:
cluster_map = pd.DataFrame()
#cluster_map['userID'] = X.index.values
cluster_map['userID'] = hs['userID'] 
cluster_map['cluster'] = kmeans.labels_
cluster_map['cluster'].head()

0    0
1    5
2    0
3    2
4    0
Name: cluster, dtype: int32

In [150]:
ls1=[]
ls2=[]
ls3=[]
ls4=[]
ls5=[]
ls6=[]
ls7=[]
ls8=[]
ls9=[]
ls10=[]
ls11=[]
ls12=[]
ls13=[]
lsn = {}
j=0
rs1=list(cluster_map['userID']);
# for i in cluster_map['cluster']:
#     if i in lsn:
#         lsn[i].append(rs[j])
#     else:
#         lsn[i] = [rs[j]]
#     j += 1

for i in cluster_map['cluster']:
    if(i==1):
        ls1.append(rs1[j]);
    if(i==2):
        ls2.append(rs1[j]);
    if(i==3):
        ls3.append(rs1[j]);
    if(i==4):
        ls4.append(rs1[j]);
    if(i==5):
        ls5.append(rs1[j]);
    if(i==6):
        ls6.append(rs1[j]);
    if(i==7):
        ls7.append(rs1[j]);
    if(i==8):
        ls8.append(rs1[j]);
    if(i==9):
        ls9.append(rs1[j]);
    if(i==10):
        ls10.append(rs1[j]);
    if(i==11):
        ls11.append(rs1[j]);
    if(i==12):
        ls12.append(rs1[j]);
    if(i==13):
        ls13.append(rs1[j])
        
    j=j+1;
k=0    

print(ls5)


['U1002', 'U1010', 'U1014', 'U1020', 'U1022', 'U1026', 'U1031', 'U1032', 'U1033', 'U1034', 'U1037', 'U1041', 'U1049', 'U1061', 'U1065', 'U1066', 'U1070', 'U1072', 'U1079', 'U1081', 'U1083', 'U1086', 'U1087', 'U1089', 'U1090', 'U1091', 'U1096', 'U1100', 'U1101', 'U1105', 'U1116', 'U1124', 'U1130', 'U1132', 'U1134', 'U1136', 'U1138']


In [151]:
rs1=list(rating_final_df['userID'])
rs=list(rating_final_df['placeID'])
js=list(rating_final_df['rating'])
lsn={}
lsn[rs1[0]]={str(rs[0]):js[0]}
kr=0
for i in rs1:
    lsn[i]={str(rs[kr]):js[kr]}
    kr=kr+1


lsn


{'U1001': {'135051': 1},
 'U1002': {'135085': 1},
 'U1003': {'135059': 2},
 'U1004': {'132958': 2},
 'U1005': {'135032': 2},
 'U1006': {'132884': 2},
 'U1007': {'135038': 1},
 'U1008': {'135054': 1},
 'U1009': {'135079': 1},
 'U1010': {'135076': 1},
 'U1011': {'132715': 1},
 'U1012': {'135018': 2},
 'U1013': {'135060': 1},
 'U1014': {'135082': 0},
 'U1015': {'132877': 1},
 'U1016': {'135062': 2},
 'U1017': {'134983': 0},
 'U1018': {'135108': 1},
 'U1019': {'135086': 0},
 'U1020': {'135109': 2},
 'U1021': {'132715': 2},
 'U1022': {'135063': 2},
 'U1023': {'132733': 2},
 'U1024': {'135030': 1},
 'U1025': {'132830': 2},
 'U1026': {'132665': 2},
 'U1027': {'135062': 1},
 'U1028': {'132613': 2},
 'U1029': {'132922': 1},
 'U1030': {'134996': 0},
 'U1031': {'132668': 0},
 'U1032': {'135032': 0},
 'U1033': {'135030': 2},
 'U1034': {'135035': 1},
 'U1035': {'134986': 2},
 'U1036': {'135045': 2},
 'U1037': {'132854': 1},
 'U1038': {'132862': 2},
 'U1039': {'132665': 1},
 'U1040': {'135019': 1},


In [152]:
rs=list(rating_final_df['placeID'])
rs1=list(rating_final_df['userID'])
lsn={}
k=0

for i in rs:
    if i in lsn:
        lsn[i].append(rs1[k])
    else:
        lsn[i]=[rs1[k]]
    k=k+1
lsn


{132560: ['U1067', 'U1082', 'U1050', 'U1087'],
 132561: ['U1026', 'U1129', 'U1065', 'U1130'],
 132564: ['U1060', 'U1080', 'U1039', 'U1011'],
 132572: ['U1108',
  'U1055',
  'U1083',
  'U1134',
  'U1007',
  'U1092',
  'U1033',
  'U1091',
  'U1090',
  'U1013',
  'U1061',
  'U1046',
  'U1112',
  'U1006',
  'U1073'],
 132583: ['U1044', 'U1118', 'U1035', 'U1119'],
 132584: ['U1067', 'U1103', 'U1107', 'U1123', 'U1063', 'U1050'],
 132594: ['U1082', 'U1123', 'U1050', 'U1087', 'U1068'],
 132608: ['U1070', 'U1082', 'U1123', 'U1080', 'U1050', 'U1043'],
 132609: ['U1070', 'U1082', 'U1063', 'U1050', 'U1043'],
 132613: ['U1103', 'U1070', 'U1082', 'U1123', 'U1028', 'U1043'],
 132626: ['U1026', 'U1065', 'U1039', 'U1130'],
 132630: ['U1068', 'U1067', 'U1103', 'U1082', 'U1028', 'U1043'],
 132654: ['U1026', 'U1129', 'U1065', 'U1130'],
 132660: ['U1107', 'U1080', 'U1063', 'U1087', 'U1068'],
 132663: ['U1068', 'U1103', 'U1031', 'U1082', 'U1028', 'U1087'],
 132665: ['U1031', 'U1026', 'U1065', 'U1129', 'U103

In [153]:
rs11=list(rating_final_df['placeID'])
rs111=list(rating_final_df['rating'])
lsn1={}
k1=0
for i in rs11:
    if i in lsn1:
        lsn1[i].append(rs111[k1])
    else:
        lsn1[i]=[rs111[k1]]
    k1=k1+1
lsn1


{132560: [1, 0, 0, 1],
 132561: [1, 1, 1, 0],
 132564: [2, 1, 1, 1],
 132572: [1, 2, 2, 0, 1, 1, 1, 2, 1, 1, 1, 1, 0, 1, 0],
 132583: [1, 0, 1, 2],
 132584: [2, 1, 2, 1, 2, 0],
 132594: [0, 1, 0, 1, 1],
 132608: [2, 0, 1, 2, 0, 1],
 132609: [1, 0, 1, 0, 1],
 132613: [2, 1, 0, 1, 2, 1],
 132626: [2, 1, 2, 0],
 132630: [1, 1, 1, 1, 2, 1],
 132654: [0, 0, 1, 0],
 132660: [2, 2, 2, 1, 0],
 132663: [1, 1, 0, 0, 0, 1],
 132665: [0, 2, 1, 0, 1],
 132667: [1, 1, 1, 2],
 132668: [0, 2, 1],
 132706: [0, 0, 2, 1],
 132715: [0, 2, 1, 1],
 132717: [0, 2, 2],
 132723: [1, 2, 1, 1, 2, 2, 1, 2, 2, 0, 2, 1],
 132732: [0, 1, 0, 0, 1, 0, 2, 1],
 132733: [1, 2, 2, 2, 1, 2, 1, 0, 1, 1],
 132740: [0, 1, 1, 2, 0, 0, 1, 1],
 132754: [2, 1, 2, 2, 2, 1, 2, 1, 0, 0, 2, 2, 2],
 132755: [2, 2, 2, 2, 1],
 132766: [1, 0, 1],
 132767: [2, 0, 1, 0, 2, 2],
 132768: [1, 2, 0, 2, 1, 2, 2, 2, 2, 2],
 132773: [2, 1, 2, 1],
 132825: [2,
  2,
  1,
  1,
  2,
  2,
  1,
  2,
  1,
  2,
  2,
  2,
  0,
  1,
  2,
  1,
  0,
  0,
  2

In [154]:
rsts={}
for i in lsn1:
    if(i in rsts):
        rsts[i].append(sum(lsn1[i])/len(lsn1[i]));
    else:
        rsts[i]=[sum(lsn1[i])/len(lsn1[i])];
rsts


{132560: [0.5],
 132561: [0.75],
 132564: [1.25],
 132572: [1.0],
 132583: [1.0],
 132584: [1.3333333333333333],
 132594: [0.6],
 132608: [1.0],
 132609: [0.6],
 132613: [1.1666666666666667],
 132626: [1.25],
 132630: [1.1666666666666667],
 132654: [0.25],
 132660: [1.4],
 132663: [0.5],
 132665: [0.8],
 132667: [1.25],
 132668: [1.0],
 132706: [0.75],
 132715: [1.0],
 132717: [1.3333333333333333],
 132723: [1.4166666666666667],
 132732: [0.625],
 132733: [1.3],
 132740: [0.75],
 132754: [1.4615384615384615],
 132755: [1.8],
 132766: [0.6666666666666666],
 132767: [1.1666666666666667],
 132768: [1.6],
 132773: [1.5],
 132825: [1.28125],
 132830: [0.9166666666666666],
 132834: [1.0],
 132845: [0.8],
 132846: [1.4],
 132847: [1.1666666666666667],
 132851: [1.4285714285714286],
 132854: [1.3333333333333333],
 132856: [0.8571428571428571],
 132858: [0.8],
 132861: [1.4285714285714286],
 132862: [1.3888888888888888],
 132866: [1.4],
 132869: [1.1666666666666667],
 132870: [0.6],
 132872: [0

In [155]:
def ratingofall(lists):
    rat1=[]
    jj=0
    r1=list(rating_final_df['userID'])   
    rt=list(rating_final_df['rating'])
    for i in lists:
        if(i in r1):
            rat1.append(rt[jj])
        jj=jj+1
    return rat1

rat1=ratingofall(ls1);
rat2=ratingofall(ls2);
rat3=ratingofall(ls3);
rat4=ratingofall(ls4);
rat5=ratingofall(ls5);
# rat6=ratingofall(ls6);
# rat7=ratingofall(ls7);
# rat8=ratingofall(ls8);
# rat9=ratingofall(ls9);
# rat10=ratingofall(ls10);
# rat11=ratingofall(ls11);
# rat12=ratingofall(ls12);
# rat13=ratingofall(ls13);

rats={}
rat11=float(sum(rat1)/len(rat1));
rat22=float(sum(rat2)/len(rat2));
rat33=float(sum(rat3)/len(rat3));
rat44=float(sum(rat4)/len(rat4));
rat55=float(sum(rat5)/len(rat5));
# rat66=float(sum(rat6)/len(rat6));
# rat77=float(sum(rat7)/len(rat7));
# rat88=float(sum(rat8)/len(rat8));
# rat99=float(sum(rat9)/len(rat9));
# rat100=float(sum(rat10)/len(rat10));
# rat111=1.0
# rat112=float(sum(rat12)/len(rat12));
# rat113=0.0

ratings=[rat11,rat22,rat33,rat44,rat55]

for i in range(1,6):
    rats[i]=ratings[i-1]
    
final=sorted(rats.items(), key=lambda x: x[1])
final.reverse()
print("All Clusters based upon rating")
final



All Clusters based upon rating


[(4, 2.0),
 (1, 1.3333333333333333),
 (2, 1.2857142857142858),
 (3, 1.2258064516129032),
 (5, 1.054054054054054)]

In [0]:
from recommendations_data import datasets
from math import sqrt

In [0]:
dataset=datasets

In [158]:
dataset

{'U1001': {'135032': 3, '135038': 3, '135058': 1, '135076': 2, '135084': 2},
 'U1002': {'135032': 1, '135038': 2, '135058': 2, '135076': 3, '135084': 1},
 'U1003': {'135031': 1, '135032': 2, '135058': 2, '135076': 3, '135084': 3},
 'U1004': {'135032': 3, '135038': 3, '135058': 1, '135076': 3, '135084': 3},
 'U1005': {'135032': 2, '135038': 2, '135058': 1, '135076': 3, '135084': 2},
 'U1006': {'135032': 3, '135058': 1, '135076': 1, '135084': 2},
 'U1007': {'135032': 3, '135033': 1, '135058': 1, '135076': 2, '135084': 2},
 'U1008': {'135032': 3, '135038': 3, '135058': 3, '135076': 3, '135084': 1},
 'U1009': {'135032': 1, '135039': 3, '135058': 1, '135076': 1, '135084': 2},
 'U1010': {'135032': 3, '135058': 3, '135076': 3, '135084': 1},
 'U1011': {'135015': 2, '135018': 2, '135060': 3},
 'U1012': {'135032': 3, '135038': 2, '135058': 2, '135076': 2, '135083': 1},
 'U1013': {'135013': 3, '135015': 2, '135018': 1, '135028': 2, '135060': 2},
 'U1014': {'135015': 2, '135032': 1, '135034': 3, '

In [159]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
# from recommendations_list import recommendation_lists
# from recommendations_data import reclists
from test_data import tests
from train_data import trains
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

def similarity_score(person1,person2):
    both_viewed = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_viewed[item] = 1

        # Conditions to check they both have an common rating items	
        if len(both_viewed) == 0:
            return 0

        # Finding Euclidean distance 
        sum_of_eclidean_distance = []

        for item in dataset[person1]:
            if item in dataset[person2]:
                sum_of_eclidean_distance.append(pow(dataset[person1][item] - dataset[person2][item],2))
        sum_of_eclidean_distance = sum(sum_of_eclidean_distance)

        return 1/(1+sqrt(sum_of_eclidean_distance))



def pearson_correlation(person1,person2):
    both_rated = {}
    for item in dataset[person1]:
        if item in dataset[person2]:
            both_rated[item] = 1

    number_of_ratings = len(both_rated)	
    if number_of_ratings == 0:
        return 0

    person1_preferences_sum = sum([dataset[person1][item] for item in both_rated])
    person2_preferences_sum = sum([dataset[person2][item] for item in both_rated])

    # Sum up the squares of preferences of each user
    person1_square_preferences_sum = sum([pow(dataset[person1][item],2) for item in both_rated])
    person2_square_preferences_sum = sum([pow(dataset[person2][item],2) for item in both_rated])

  
    product_sum_of_both_users = sum([dataset[person1][item] * dataset[person2][item] for item in both_rated])

    numerator_value = product_sum_of_both_users - (person1_preferences_sum*person2_preferences_sum/number_of_ratings)
    denominator_value = sqrt((person1_square_preferences_sum - pow(person1_preferences_sum,2)/number_of_ratings) * (person2_square_preferences_sum -pow(person2_preferences_sum,2)/number_of_ratings))
    if denominator_value == 0:
        return 0
    else:
        r = numerator_value/denominator_value
        return abs(r)

    
    
    
    
def most_similar_users(person,number_of_users):
    scores = [(pearson_correlation(person,other_person),other_person) for other_person in dataset if  other_person != person ]
    scores.sort()
    scores.reverse()
    return scores[0:number_of_users]



def user_reommendations(person):
    totals = {}
    simSums = {}
    sim =[]
    for other in dataset:
        if other == person:
            continue
        sim = pearson_correlation(person,other)
        
        if sim <=0: 
            continue
        for item in dataset[other]:
            if item not in dataset[person] or dataset[person][item] == 0:
                totals.setdefault(item,0)
#                 print("Sim",sim,person,other)
                totals[item] += dataset[other][item]* sim
                simSums.setdefault(item,0)
                simSums[item]+= sim            

    rankings = [(total/simSums[item],item) for item,total in totals.items()]
    rankings.sort()
    
    rankings.reverse()
#     print("rankings", rankings)
    recommendation_list = [recommend_item for score,recommend_item in rankings]
    
    return recommendation_list


print("SIMILARITY SCORES\n")
print(similarity_score('U1004','U1001'),"\n")
print("MOST SIMILAR USERS\n")
print(most_similar_users('U1002',3))
print("\nPEARSON_CORRELATION\n")
print(pearson_correlation('U1002','U1001'))
print("\nUSER Recommendation\n")
print(user_reommendations('U1002'))

def intersect(b1, b2):
    return list(set(b1).intersection(b2))

# user_reommendations('U1002')
# total = 0
# zero_count = 0
# for user in dataset:
#     result = user_reommendations(user)
#     if len(result) == 0:
#         zero_count += 1
#     else:
#         print(user, intersect(result, list(dataset[user].keys())))
#     total += 1
# print(zero_count, total)


SIMILARITY SCORES

0.4142135623730951 

MOST SIMILAR USERS

[(1.0, 'U1022'), (1.0, 'U1016'), (0.9999999999999987, 'U1020')]

PEARSON_CORRELATION

0.2857142857142859

USER Recommendation

['135089', '135085', '135073', '135039', '135034', '135030', '134996', '132668', '135118', '135083', '135033', '135031']


In [0]:
from collections import Counter

In [0]:
def get_liked_places(user):
    return [place for place in dataset[user] if dataset[user][place] >= 2]


def calculate_recall(user):
    liked_places = get_liked_places(user)
    predicted_places = user_reommendations(user)
    if len(liked_places) == 0:
        return 0.0
    else:
        return len(intersect(liked_places, predicted_places)) / len(liked_places)

def calculate_precision(user, liked_places_count):
    predicted_places = user_reommendations(user)
    if len(predicted_places) == 0:
        return 0.0
    else:
        return liked_places_count / len(predicted_places)

def avgreprecision(dataset):
    avg_precision_sum=0;
    j=0;
    for user in dataset:
        avg_precision_sum+= calculate_precision(user,.2);
        j=j+1
    avg_precision=float(avg_precision_sum/(len(user)))
    return avg_precisions

def avgrecall(dataset):
    avg_recall_sum=0;
    for user in dataset:
        avg_recall_sum+= calculate_recall(user);
    avg_recall=float(avg_recall_sum/(len(user)))
    return avg_recalls

def calculate_mean_average_precision(recalls):
    d = dict(Counter(recalls))
    avg = 0
    n = len(recalls)
    for recall in d:
        avg += d[recall] / n
    return avgs

def list_of_precision(tests):
    prelist=[]
    for user in tests:
        prelist.append(calculate_precision(user,3));
        
def list_of_recall(tests):
    reclist=[]
    for user in tests:
        reclist.append(calculate_recalls(user));
        

In [162]:
print("ALL Test Data")
for i in tests:
        print(i,tests.values())

ALL Test Data
U1001 dict_values([{'135058': 1, '135038': 3, '135032': 3, '135084': 2, '135076': 2}, {'135058': 2, '135038': 2, '135032': 1, '135084': 1, '135076': 3}, {'135058': 2, '135031': 1, '135032': 2, '135084': 3, '135076': 3}, {'135058': 1, '135038': 3, '135032': 3, '135084': 3, '135076': 3}, {'135058': 1, '135038': 2, '135032': 2, '135084': 2, '135076': 3}, {'135058': 1, '135032': 3, '135084': 2, '135076': 1}, {'135058': 1, '135033': 1, '135032': 3, '135084': 2, '135076': 2}, {'135058': 3, '135038': 3, '135032': 3, '135084': 1, '135076': 3}, {'135058': 1, '135039': 3, '135032': 1, '135084': 2, '135076': 1}, {'135058': 3, '135032': 3, '135084': 1, '135076': 3}, {'135015': 2, '135018': 2, '135060': 3}, {'135058': 2, '135038': 2, '135032': 3, '135083': 1, '135076': 2}, {'135015': 2, '135028': 2, '135060': 2, '135013': 3, '135018': 1}, {'135015': 2, '135034': 3, '135060': 3, '135039': 3, '135032': 1}, {'135015': 2, '135018': 2, '135060': 1}, {'135058': 2, '135038': 2, '135032': 1, 

In [163]:

avg_recall=avgrecall(tests)
avg_recall


0.7972556442597871

In [164]:

avg_precision=avgreprecision(tests)
avg_precision


0.8532526879527471

In [0]:
prelist=[]
reclist=[]

# cal_pre_re()
for user in dataset:
        prelist.append(calculate_precision(user,3));

for user in dataset:
        reclist.append(calculate_recall(user));


In [166]:
print("List of Precision of Every User ")
for i in range(len(prelist)):
    if(i==20):
        break;
    else:
        print(prelist[i])

List of Precision of Every User 
0.23076923076923078
0.25
0.3
0.2
0.21428571428571427
0.3333333333333333
0.3333333333333333
0.5
0.375
0.42857142857142855
0.6
0.23076923076923078
0.0
0.75
0.6
0.3333333333333333
0.375
0.0
0.0
0.3


In [167]:
print("List of Recalls of Every User")
for i in range(len(reclists)):
    if(i==20):
        break;
    else:
        print(reclists[i])

List of Recalls of Every User
0.13076923076923078
0.2076923076923078
0.13076923076923078
0.23076923076923078
0.3142857142857143
0.23134567837883904
0.2456333333333333
0.5307692307692302
0.2753076923076923
0.32857142857142857
0.1
0.23076923076923078
0.0
0.35
0.23076923076923078
0.2333333333333333
0.375
0.0
0.1
0.2


In [0]:

cal_m_a_p=calculate_mean_average_precision(prelist)

In [169]:
print("Mean Average Precision=",cal_m_a_p)

Mean Average Precision= 0.8774525678920471


In [170]:
from sklearn.model_selection import train_test_split
columns=['userID','placeID','rating','food_rating', 'service_rating']
df = pd.DataFrame(rating_final_df,columns=columns)
df.drop(['food_rating', 'service_rating'], axis=1)
df.head()



,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [171]:
datasets

{'U1001': {'135032': 3, '135038': 3, '135058': 1, '135076': 2, '135084': 2},
 'U1002': {'135032': 1, '135038': 2, '135058': 2, '135076': 3, '135084': 1},
 'U1003': {'135031': 1, '135032': 2, '135058': 2, '135076': 3, '135084': 3},
 'U1004': {'135032': 3, '135038': 3, '135058': 1, '135076': 3, '135084': 3},
 'U1005': {'135032': 2, '135038': 2, '135058': 1, '135076': 3, '135084': 2},
 'U1006': {'135032': 3, '135058': 1, '135076': 1, '135084': 2},
 'U1007': {'135032': 3, '135033': 1, '135058': 1, '135076': 2, '135084': 2},
 'U1008': {'135032': 3, '135038': 3, '135058': 3, '135076': 3, '135084': 1},
 'U1009': {'135032': 1, '135039': 3, '135058': 1, '135076': 1, '135084': 2},
 'U1010': {'135032': 3, '135058': 3, '135076': 3, '135084': 1},
 'U1011': {'135015': 2, '135018': 2, '135060': 3},
 'U1012': {'135032': 3, '135038': 2, '135058': 2, '135076': 2, '135083': 1},
 'U1013': {'135013': 3, '135015': 2, '135018': 1, '135028': 2, '135060': 2},
 'U1014': {'135015': 2, '135032': 1, '135034': 3, '

In [172]:
cluster_map.head()

,userID,cluster
0,U1001,0
1,U1002,5
2,U1003,0
3,U1004,2
4,U1005,0


#Check for null value
geoplaces2_df.isnull().sum()

In [173]:
geoplaces2_df.shape


(130, 21)

In [174]:
geoplaces2_df.head(1)

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none


In [175]:
for col in geoplaces2_df.columns:
    if geoplaces2_df[col].dtype == object:         
            print(col, (geoplaces2_df[col].str.contains('\?') == True).sum()) 

the_geom_meter 0
name 0
address 27
city 18
state 18
country 28
fax 130
zip 74
alcohol 0
smoking_area 0
dress_code 0
accessibility 0
price 0
url 116
Rambience 0
franchise 0
area 0
other_services 0


In [0]:
columns_with_na_vals= ['address','city','state','country','fax','zip','url']

In [0]:
for col in columns_with_na_vals:
    geoplaces2_df[col] = (geoplaces2_df[col].replace({'?': np.NaN})) 

In [178]:
geoplaces2_df.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,NaN,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,NaN,...,Wine-Beer,only at bar,informal,partially,medium,NaN,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,NaN,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,NaN,...,No_Alcohol_Served,permitted,informal,completely,medium,NaN,familiar,t,closed,none


In [179]:

geoplaces2_df

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,NaN,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,NaN,...,Wine-Beer,only at bar,informal,partially,medium,NaN,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,NaN,NaN,...,No_Alcohol_Served,none,informal,completely,low,NaN,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,NaN,...,No_Alcohol_Served,permitted,informal,completely,medium,NaN,familiar,t,closed,none
5,135040,22.135617,-100.969709,0101000020957F00001B552189B84A58C15A2AAEFD2CA2...,Restaurant los Compadres,Camino a Simon Diaz 155 Centro,San Luis Potosi,SLP,Mexico,NaN,...,Wine-Beer,none,informal,no_accessibility,high,NaN,familiar,f,closed,none
6,132732,23.754357,-99.171288,0101000020957F00008A20E615808157C16272FECBF84F...,Taqueria EL amigo,Calle Mezquite Fracc Framboyanes,Cd Victoria,Tamaulipas,Mexico,NaN,...,No_Alcohol_Served,none,casual,completely,low,NaN,familiar,f,open,none
7,132875,22.149901,-100.993779,0101000020957F00008A2A0747DE4758C11EB31D2A31A8...,shi ro ie,NaN,NaN,NaN,NaN,NaN,...,Wine-Beer,section,informal,no_accessibility,high,NaN,familiar,t,open,Internet
8,132609,23.760268,-99.165865,0101000020957F0000A478418BBA8057C133851EB22C4E...,Pollo_Frito_Buenos_Aires,tampico,victoria,Tamaulipas,Mexico,NaN,...,No_Alcohol_Served,not permitted,informal,completely,low,NaN,quiet,t,closed,none
9,135082,22.151448,-100.915099,0101000020957F0000A29FAF95CD4958C1FEEEBB73A991...,la Estrella de Dimas,Villa de Pozos 192 Villa de Pozos,San Luis Potosi,SLP,Mexico,NaN,...,No_Alcohol_Served,none,informal,no_accessibility,medium,NaN,familiar,f,closed,none


#### Concatenate All restaurent file together of common column placeID

In [180]:
restaurant_all = np.concatenate((chefmozaccepts_df.placeID, chefmozcuisine_df.placeID, chefmozhours4_df.placeID, chefmozparking_df.placeID, geoplaces2_df.placeID))
restaurant_all = np.sort(np.unique(restaurant_all) ) # All the unique placeID's
print(len(restaurant_all))

938


In [181]:
restaurant_all

array([132001, 132002, 132003, 132004, 132005, 132006, 132007, 132008,
       132009, 132010, 132012, 132013, 132014, 132015, 132016, 132017,
       132018, 132019, 132020, 132021, 132022, 132023, 132024, 132025,
       132026, 132028, 132030, 132031, 132083, 132087, 132092, 132094,
       132096, 132097, 132098, 132100, 132101, 132102, 132103, 132105,
       132106, 132107, 132108, 132109, 132111, 132114, 132115, 132116,
       132118, 132119, 132120, 132121, 132125, 132126, 132127, 132128,
       132130, 132131, 132132, 132133, 132136, 132137, 132138, 132145,
       132146, 132147, 132148, 132155, 132156, 132157, 132159, 132160,
       132161, 132162, 132163, 132164, 132165, 132166, 132167, 132171,
       132174, 132175, 132177, 132180, 132182, 132184, 132186, 132187,
       132191, 132192, 132201, 132203, 132204, 132206, 132207, 132208,
       132209, 132210, 132211, 132212, 132213, 132214, 132215, 132216,
       132217, 132218, 132220, 132221, 132223, 132225, 132226, 132228,
      

Note: Here we concatenate all restaurant files according to their placeID, it will give all common placeID which is present in all five .csv file. Finally, we get total 938 placeID which is unique.

### Explore the user item rating files
#### rating_final.csv 

In [182]:
rating_final_df.columns

Index(['userID', 'placeID', 'rating', 'food_rating', 'service_rating'], dtype='object')

In [183]:
rating_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
userID            1161 non-null object
placeID           1161 non-null int64
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(4), object(1)
memory usage: 45.4+ KB


In [184]:
rating_final_df.shape

(1161, 5)

In [185]:
rating_final_df.isnull().sum()

userID            0
placeID           0
rating            0
food_rating       0
service_rating    0
dtype: int64

In [186]:
rating_final_df[['rating','food_rating','service_rating']].describe()

,rating,food_rating,service_rating
count,1161.000000,1161.000000,1161.000000
mean,1.199828,1.215332,1.090439
std,0.773282,0.792294,0.790844
min,0.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000
50%,1.000000,1.000000,1.000000
75%,2.000000,2.000000,2.000000
max,2.000000,2.000000,2.000000


In [187]:
rating_final_df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [188]:
len(rating_final_df.placeID.unique())

130